# 109 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [3]:
no_of_patients=109

In [4]:
train=sorted(glob('files4/*.edf'))
train=train[:no_of_patients]
train

['files4/S001R06.edf',
 'files4/S002R06.edf',
 'files4/S003R06.edf',
 'files4/S004R06.edf',
 'files4/S005R06.edf',
 'files4/S006R06.edf',
 'files4/S007R06.edf',
 'files4/S008R06.edf',
 'files4/S009R06.edf',
 'files4/S010R06.edf',
 'files4/S011R06.edf',
 'files4/S012R06.edf',
 'files4/S013R06.edf',
 'files4/S014R06.edf',
 'files4/S015R06.edf',
 'files4/S016R06.edf',
 'files4/S017R06.edf',
 'files4/S018R06.edf',
 'files4/S019R06.edf',
 'files4/S020R06.edf',
 'files4/S021R06.edf',
 'files4/S022R06.edf',
 'files4/S023R06.edf',
 'files4/S024R06.edf',
 'files4/S025R06.edf',
 'files4/S026R06.edf',
 'files4/S027R06.edf',
 'files4/S028R06.edf',
 'files4/S029R06.edf',
 'files4/S030R06.edf',
 'files4/S031R06.edf',
 'files4/S032R06.edf',
 'files4/S033R06.edf',
 'files4/S034R06.edf',
 'files4/S035R06.edf',
 'files4/S036R06.edf',
 'files4/S037R06.edf',
 'files4/S038R06.edf',
 'files4/S039R06.edf',
 'files4/S040R06.edf',
 'files4/S041R06.edf',
 'files4/S042R06.edf',
 'files4/S043R06.edf',
 'files4/S0

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(1308000, 163391, 1308000, 163391)

In [10]:
print(xtest.iloc[:,-1].values)

[-5.8e-05 -3.7e-05 -1.0e-06 ... -2.6e-05 -1.7e-05 -5.0e-06]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000045,0.000128,0.000290,0.000177,0.000145,0.000001,0.000156,0.000072,0.000023,0.000128,...,0.000040,0.000062,0.000023,0.000045,-0.000007,0.000083,0.000036,0.000061,0.000064,0.000045
1307996,0.000043,0.000117,0.000287,0.000166,0.000133,0.000051,0.000139,0.000062,0.000010,0.000135,...,0.000039,0.000064,0.000024,0.000033,-0.000011,0.000079,0.000034,0.000058,0.000060,0.000047
1307997,0.000043,0.000092,0.000284,0.000149,0.000106,0.000031,0.000121,0.000050,-0.000010,0.000103,...,0.000039,0.000060,0.000018,0.000024,-0.000018,0.000076,0.000026,0.000049,0.000047,0.000037
1307998,0.000052,0.000086,0.000277,0.000142,0.000096,0.000022,0.000116,0.000046,-0.000010,0.000099,...,0.000021,0.000031,-0.000009,0.000001,-0.000045,0.000051,-0.000001,0.000025,0.000012,0.000014


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000045,0.000128,0.000290,0.000177,0.000145,0.000001,0.000156,0.000072,0.000023,0.000128,...,0.000040,0.000062,0.000023,0.000045,-0.000007,0.000083,0.000036,0.000061,0.000064,0.000045
1307996,0.000043,0.000117,0.000287,0.000166,0.000133,0.000051,0.000139,0.000062,0.000010,0.000135,...,0.000039,0.000064,0.000024,0.000033,-0.000011,0.000079,0.000034,0.000058,0.000060,0.000047
1307997,0.000043,0.000092,0.000284,0.000149,0.000106,0.000031,0.000121,0.000050,-0.000010,0.000103,...,0.000039,0.000060,0.000018,0.000024,-0.000018,0.000076,0.000026,0.000049,0.000047,0.000037
1307998,0.000052,0.000086,0.000277,0.000142,0.000096,0.000022,0.000116,0.000046,-0.000010,0.000099,...,0.000021,0.000031,-0.000009,0.000001,-0.000045,0.000051,-0.000001,0.000025,0.000012,0.000014


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
40875/40875 [==============================] - 34s 831us/step - loss: 3.1358 - val_loss: 3.3545
Epoch 2/10
40875/40875 [==============================] - 194s 5ms/step - loss: 2.6282 - val_loss: 3.2771
Epoch 3/10
40875/40875 [==============================] - 216s 5ms/step - loss: 2.4791 - val_loss: 3.2644
Epoch 4/10
40875/40875 [==============================] - 110s 3ms/step - loss: 2.4296 - val_loss: 3.3183
Epoch 5/10
40875/40875 [==============================] - 127s 3ms/step - loss: 2.3977 - val_loss: 3.2919
Epoch 6/10
40875/40875 [==============================] - 72s 2ms/step - loss: 2.3754 - val_loss: 3.3805
Epoch 7/10
40875/40875 [==============================] - 77s 2ms/step - loss: 2.3576 - val_loss: 3.4164
Epoch 8/10
40875/40875 [==============================] - 1084s 27ms/step - loss: 2.3432 - val_loss: 3.3356
Epoch 9/10
40875/40875 [==============================] - 333s 8ms/step - loss: 2.3306 - val_loss: 3.4074
Epoch 10/10
40875/40875 [====================

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

5106/5106 [==============================] - 2s 373us/step
              precision    recall  f1-score   support

           0       0.12      0.09      0.10      1499
           1       0.05      0.03      0.03      1499
           2       0.17      0.20      0.18      1499
           3       0.72      0.34      0.46      1499
           4       0.68      0.66      0.67      1499
           5       0.37      0.30      0.33      1499
           6       0.41      0.44      0.43      1499
           7       0.46      0.49      0.47      1499
           8       0.40      0.60      0.48      1499
           9       0.40      0.25      0.30      1499
          10       0.27      0.31      0.29      1499
          11       0.17      0.41      0.24      1499
          12       0.17      0.04      0.06      1499
          13       0.06      0.06      0.06      1499
          14       0.07      0.01      0.02      1499
          15       0.34      0.28      0.31      1499
          16       0.1

## 0-16

In [19]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [20]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000045,0.000128,0.000290,0.000177,0.000145,0.000001,0.000156,0.000072,0.000023,0.000128,...,0.000040,0.000062,0.000023,0.000045,-0.000007,0.000083,0.000036,0.000061,0.000064,0.000045
1307996,0.000043,0.000117,0.000287,0.000166,0.000133,0.000051,0.000139,0.000062,0.000010,0.000135,...,0.000039,0.000064,0.000024,0.000033,-0.000011,0.000079,0.000034,0.000058,0.000060,0.000047
1307997,0.000043,0.000092,0.000284,0.000149,0.000106,0.000031,0.000121,0.000050,-0.000010,0.000103,...,0.000039,0.000060,0.000018,0.000024,-0.000018,0.000076,0.000026,0.000049,0.000047,0.000037
1307998,0.000052,0.000086,0.000277,0.000142,0.000096,0.000022,0.000116,0.000046,-0.000010,0.000099,...,0.000021,0.000031,-0.000009,0.000001,-0.000045,0.000051,-0.000001,0.000025,0.000012,0.000014


In [21]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
40875/40875 [==============================] - 36s 864us/step - loss: 2.3679 - val_loss: 3.1601
Epoch 2/10
40875/40875 [==============================] - 40s 980us/step - loss: 1.7040 - val_loss: 3.2252
Epoch 3/10
40875/40875 [==============================] - 39s 960us/step - loss: 1.5331 - val_loss: 3.0932
Epoch 4/10
40875/40875 [==============================] - 37s 916us/step - loss: 1.4436 - val_loss: 3.2501
Epoch 5/10
40875/40875 [==============================] - 38s 929us/step - loss: 1.3935 - val_loss: 3.2252
Epoch 6/10
40875/40875 [==============================] - 38s 926us/step - loss: 1.3598 - val_loss: 3.3637
Epoch 7/10
40875/40875 [==============================] - 37s 917us/step - loss: 1.3343 - val_loss: 3.3413
Epoch 8/10
40875/40875 [==============================] - 41s 1ms/step - loss: 1.3151 - val_loss: 3.4990
Epoch 9/10
40875/40875 [==============================] - 43s 1ms/step - loss: 1.2987 - val_loss: 3.2693
Epoch 10/10
40875/40875 [================

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

5106/5106 [==============================] - 2s 420us/step
              precision    recall  f1-score   support

           0       0.23      0.40      0.29      1499
           1       0.10      0.13      0.11      1499
           2       0.29      0.17      0.21      1499
           3       0.69      0.41      0.51      1499
           4       0.84      0.81      0.82      1499
           5       0.55      0.60      0.57      1499
           6       0.69      0.70      0.69      1499
           7       0.70      0.40      0.51      1499
           8       0.48      0.64      0.54      1499
           9       0.42      0.65      0.51      1499
          10       0.50      0.28      0.35      1499
          11       0.49      0.79      0.60      1499
          12       0.30      0.42      0.35      1499
          13       0.20      0.24      0.21      1499
          14       0.16      0.10      0.12      1499
          15       0.62      0.51      0.56      1499
          16       0.2

## 0-32

In [25]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [26]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63563/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [27]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
40875/40875 [==============================] - 46s 1ms/step - loss: 1.5920 - val_loss: 2.6741
Epoch 2/10
40875/40875 [==============================] - 48s 1ms/step - loss: 0.8376 - val_loss: 2.6282
Epoch 3/10
40875/40875 [==============================] - 53s 1ms/step - loss: 0.6869 - val_loss: 2.5394
Epoch 4/10
40875/40875 [==============================] - 50s 1ms/step - loss: 0.6111 - val_loss: 2.4157
Epoch 5/10
40875/40875 [==============================] - 52s 1ms/step - loss: 0.5653 - val_loss: 2.3471
Epoch 6/10
40875/40875 [==============================] - 55s 1ms/step - loss: 0.5337 - val_loss: 2.4478
Epoch 7/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.5086 - val_loss: 2.4176
Epoch 8/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.4879 - val_loss: 2.3727
Epoch 9/10
40875/40875 [==============================] - 55s 1ms/step - loss: 0.4698 - val_loss: 2.3888
Epoch 10/10
40875/40875 [==============================

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

5106/5106 [==============================] - 3s 663us/step
              precision    recall  f1-score   support

           0       0.74      0.63      0.68      1499
           1       0.26      0.18      0.21      1499
           2       0.62      0.81      0.70      1499
           3       0.76      0.30      0.43      1499
           4       0.85      0.95      0.90      1499
           5       0.90      0.91      0.90      1499
           6       0.91      0.70      0.79      1499
           7       0.70      0.57      0.63      1499
           8       0.54      0.82      0.65      1499
           9       0.85      0.52      0.65      1499
          10       0.84      0.82      0.83      1499
          11       0.91      0.92      0.91      1499
          12       0.86      0.69      0.77      1499
          13       0.61      0.70      0.65      1499
          14       0.45      0.65      0.54      1499
          15       0.78      0.52      0.63      1499
          16       0.4

## 0-64

In [31]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [32]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
40875/40875 [==============================] - 62s 2ms/step - loss: 1.2391 - val_loss: 2.6627
Epoch 2/10
40875/40875 [==============================] - 69s 2ms/step - loss: 0.5587 - val_loss: 2.5208
Epoch 3/10
40875/40875 [==============================] - 64s 2ms/step - loss: 0.4044 - val_loss: 2.2964
Epoch 4/10
40875/40875 [==============================] - 66s 2ms/step - loss: 0.3271 - val_loss: 2.2771
Epoch 5/10
40875/40875 [==============================] - 65s 2ms/step - loss: 0.2807 - val_loss: 2.3602
Epoch 6/10
40875/40875 [==============================] - 62s 2ms/step - loss: 0.2488 - val_loss: 2.2493
Epoch 7/10
40875/40875 [==============================] - 75s 2ms/step - loss: 0.2253 - val_loss: 2.0812
Epoch 8/10
40875/40875 [==============================] - 76s 2ms/step - loss: 0.2062 - val_loss: 1.9723
Epoch 9/10
40875/40875 [==============================] - 64s 2ms/step - loss: 0.1909 - val_loss: 2.0480
Epoch 10/10
40875/40875 [==============================

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

5106/5106 [==============================] - 4s 689us/step
              precision    recall  f1-score   support

           0       0.90      0.79      0.84      1499
           1       0.56      0.45      0.50      1499
           2       0.73      0.85      0.78      1499
           3       0.81      0.94      0.87      1499
           4       0.96      0.99      0.97      1499
           5       0.93      0.97      0.95      1499
           6       0.92      0.87      0.89      1499
           7       0.88      0.62      0.72      1499
           8       0.72      0.85      0.78      1499
           9       0.81      0.88      0.85      1499
          10       0.68      0.72      0.70      1499
          11       0.96      0.99      0.97      1499
          12       0.92      0.82      0.87      1499
          13       0.75      0.77      0.76      1499
          14       0.78      0.80      0.79      1499
          15       0.71      0.94      0.81      1499
          16       0.8